In [7]:
import os

import json
import shutil

from PIL import Image
from sklearn.model_selection import train_test_split

# Подготовка меток до требуемого YOLOv8 формата

In [126]:
os.makedirs('part', exist_ok=True)
os.makedirs('full', exist_ok=True)

part_folder = '/train_dataset_dataset/part/labels'
full_folder = '/train_dataset_dataset/full/labels'

part_output_folder = 'dataset/part/labels'
full_output_folder = 'dataset/full/labels'

Для перевода меток парсятся исходные json файлы с метками и переводятся в txt-формат
формата:

label cx cy w h 

Где:
    
-label - метка класса;
    
-cx - центральная координата bounding_box по оси x;
    
-cy - центральная координата bounding_box по оси y;
    
-w - ширина bounding_box;
    
-h - высота bounding_box.
    
Все координаты представлены в относительном виде.

In [ ]:
def convert_and_save_to_txt(json_folder, output_folder, images_folder):
    json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]
    
    for json_file in json_files:
        with open(os.path.join(json_folder, json_file), 'r') as json_file:
            data = json.load(json_file)
            image_name = data[0]['image'].split('.')[0]
            coordinates = data[0]['annotations'][0]['coordinates']
            
            image_path = os.path.join(images_folder, f'{image_name}.jpg')
            with Image.open(image_path) as img:
                img_width, img_height = img.size
            x = coordinates['x']
            y = coordinates['y']               
            width = coordinates['width']
            height = coordinates['height']
            
            x_rel = round(x / img_width, 6) #relative coordinates for yolo8 labels format
            y_rel = round(y / img_height, 6)
            width_rel = round(width / img_width, 6)
            height_rel = round(height / img_height, 6)
            
            label = '0'
            txt_data = f'{label} {x_rel} {y_rel} {width_rel} {height_rel}'
            
            with open(os.path.join(output_folder, f'{image_name}.txt'), 'w') as txt_file:
                txt_file.write(txt_data)

convert_and_save_to_txt(part_folder, part_output_folder, '/train_dataset_dataset/part/images')
convert_and_save_to_txt(full_folder, full_output_folder, '/train_dataset_dataset/full/images')

In [127]:
images_dir = '/yolo/dataset/null/images'
labels_dir = '/yolo/dataset/null/labels'

image_files = os.listdir(images_dir)

for image_file in image_files:
    filename_no_ext = os.path.splitext(image_file)[0]
    label_path = os.path.join(labels_dir, f"{filename_no_ext}.txt")
    
    open(label_path, 'w').close()

In [129]:
def move_files(source_folder, destination_folder, files):
    os.makedirs(os.path.join(destination_folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(destination_folder, "labels"), exist_ok=True)
    for file in files:
        source_image_path = os.path.join(source_folder, 'images', file)
        source_label_path = os.path.join(source_folder, "labels", file.replace(".jpg", ".txt"))
        destination_image_path = os.path.join(destination_folder, "images", file)
        destination_label_path = os.path.join(destination_folder, "labels", file.replace(".jpg", ".txt"))
        shutil.move(source_image_path, destination_image_path)
        shutil.move(source_label_path, destination_label_path)

In [130]:
source_dir = '/yolo/dataset/without_part/train'
val_dir = "/yolo/dataset/without_part/val"

image_files = [f for f in os.listdir(source_dir + '/images') if f.endswith((".jpg"))]

image_files_train, image_files_val = train_test_split(image_files, test_size=0.15, random_state=42)

move_files(source_dir, val_dir, image_files_val)

In [131]:
source_dir = '/yolo/dataset/with_part/train'
val_dir = "/yolo/dataset/with_part/val"

image_files = [f for f in os.listdir(source_dir + '/images') if f.endswith((".jpg"))]

image_files_train, image_files_val = train_test_split(image_files, test_size=0.15, random_state=42)

move_files(source_dir, val_dir, image_files_val)